In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/ELEC_576/Final_Project/archive.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Bing_0289.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Bing_0292.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Bing_0299.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Bing_0305.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Bing_0319.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Bing_0335.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Bing_0339.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Bing_0341.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Google_0018.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Google_0103.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Google_0110.jpg  
  inflating: /content/plantsegv2/images/train/peach_brown_rot_Google_0128.jpg  
  inf

In [3]:
import os
import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from einops import rearrange
import timm
from collections import OrderedDict
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
from datetime import datetime


class PlantSegDataset(Dataset):
    """Dataset loader for PlantSeg dataset with pre-split train/val/test"""

    def __init__(self, root_dir, split='train', img_size=224, transform=None):

        self.root_dir = root_dir
        self.split = split
        self.img_size = img_size
        self.transform = transform

        # Set paths for images and masks based on split
        self.img_dir = os.path.join(root_dir, 'images', split)
        self.mask_dir = os.path.join(root_dir, 'annotations', split)

        # Get all image files
        self.image_files = sorted([f for f in os.listdir(self.img_dir)
                                   if f.endswith(('.jpg', '.jpeg', '.png'))])

        print(f"{split.upper()} dataset: {len(self.image_files)} images")
        print(f"  Images dir: {self.img_dir}")
        print(f"  Masks dir:  {self.mask_dir}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get image filename
        img_filename = self.image_files[idx]

        # Load image
        img_path = os.path.join(self.img_dir, img_filename)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask_filename = os.path.splitext(img_filename)[0] + '.png'
        mask_path = os.path.join(self.mask_dir, mask_filename)

        # Read mask as grayscale
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if mask is None:
            mask_path = os.path.join(self.mask_dir, img_filename)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if mask is None:
            raise FileNotFoundError(f"Mask not found for image: {img_filename}")

        unique_vals = np.unique(mask)

        if len(unique_vals) == 2:
            # Binary mask - normalize to 0/1
            mask = (mask == mask.max()).astype(np.float32)
        else:
            # Multi-value mask - threshold at 127 (middle of 0-255)
            mask = (mask > 127).astype(np.float32)

        # Double-check: mask should only have 0 and 1
        assert mask.min() >= 0 and mask.max() <= 1, f"Mask values out of range: {mask.min()}-{mask.max()}"

        # Apply transforms
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']

        # Convert mask to tensor and add channel dimension
        if not isinstance(mask, torch.Tensor):
            mask = torch.from_numpy(mask).float()

        if len(mask.shape) == 2:
            mask = mask.unsqueeze(0)

        return image, mask


def get_transforms(img_size=224, is_train=True):
    """Get augmentation transforms"""
    if is_train:
        return A.Compose([
            A.Resize(img_size, img_size),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(img_size, img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])


def get_dataloaders(root_dir, img_size=224, batch_size=8, num_workers=4):
    """Create train, val, and test dataloaders"""

    train_dataset = PlantSegDataset(
        root_dir=root_dir,
        split='train',
        img_size=img_size,
        transform=get_transforms(img_size, is_train=True)
    )

    val_dataset = PlantSegDataset(
        root_dir=root_dir,
        split='val',
        img_size=img_size,
        transform=get_transforms(img_size, is_train=False)
    )

    test_dataset = PlantSegDataset(
        root_dir=root_dir,
        split='test',
        img_size=img_size,
        transform=get_transforms(img_size, is_train=False)
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_loader, val_loader, test_loader

In [4]:
class DiceLoss(nn.Module):
    """Dice Loss for binary segmentation"""

    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, predictions, targets):
        # Flatten predictions and targets
        predictions = predictions.view(-1)
        targets = targets.view(-1)

        # Calculate intersection and union
        intersection = (predictions * targets).sum()
        dice = (2. * intersection + self.smooth) / (predictions.sum() + targets.sum() + self.smooth)

        return 1 - dice


class FocalLoss(nn.Module):
    """Focal Loss for addressing class imbalance"""

    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, predictions, targets):
        # Binary cross entropy
        bce_loss = F.binary_cross_entropy(predictions, targets, reduction='none')

        # Focal weight
        pt = torch.exp(-bce_loss)
        focal_weight = self.alpha * (1 - pt) ** self.gamma

        # Focal loss
        focal_loss = focal_weight * bce_loss

        return focal_loss.mean()


class BCEDiceLoss(nn.Module):
    """Combined BCE and Dice Loss"""

    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super(BCEDiceLoss, self).__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.bce = nn.BCELoss()
        self.dice = DiceLoss()

    def forward(self, predictions, targets):
        bce_loss = self.bce(predictions, targets)
        dice_loss = self.dice(predictions, targets)

        return self.bce_weight * bce_loss + self.dice_weight * dice_loss


class TverskyLoss(nn.Module):
    """Tversky Loss - generalization of Dice loss"""

    def __init__(self, alpha=0.5, beta=0.5, smooth=1.0):
        super(TverskyLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.smooth = smooth

    def forward(self, predictions, targets):
        # Flatten
        predictions = predictions.view(-1)
        targets = targets.view(-1)

        # True Positives, False Positives & False Negatives
        TP = (predictions * targets).sum()
        FP = ((1 - targets) * predictions).sum()
        FN = (targets * (1 - predictions)).sum()

        tversky = (TP + self.smooth) / (TP + self.alpha * FP + self.beta * FN + self.smooth)

        return 1 - tversky


def get_loss_function(loss_name):
    """Factory function to get loss function by name"""

    loss_functions = {
        'bce': nn.BCELoss(),
        'dice': DiceLoss(),
        'focal': FocalLoss(alpha=0.25, gamma=2.0),
        'bce_dice': BCEDiceLoss(bce_weight=0.5, dice_weight=0.5),
        'tversky': TverskyLoss(alpha=0.5, beta=0.5)
    }

    if loss_name.lower() not in loss_functions:
        raise ValueError(f"Loss function '{loss_name}' not recognized. "
                         f"Available options: {list(loss_functions.keys())}")

    return loss_functions[loss_name.lower()]

In [5]:

class PatchEmbedding(nn.Module):
    """Split image into patches and embed them"""

    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2

        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # (B, embed_dim, n_patches**0.5, n_patches**0.5)
        x = x.flatten(2)  # (B, embed_dim, n_patches)
        x = x.transpose(1, 2)  # (B, n_patches, embed_dim)
        return x


class MultiHeadAttention(nn.Module):
    """Multi-head self-attention mechanism"""

    def __init__(self, embed_dim=768, num_heads=12, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.attn_drop = nn.Dropout(dropout)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.proj_drop = nn.Dropout(dropout)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class MLP(nn.Module):
    """Feed-forward network"""

    def __init__(self, in_features, hidden_features=None, out_features=None, dropout=0.1):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features

        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class TransformerBlock(nn.Module):
    """Transformer encoder block"""

    def __init__(self, embed_dim=768, num_heads=12, mlp_ratio=4., dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = MultiHeadAttention(embed_dim, num_heads, dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = MLP(embed_dim, mlp_hidden_dim, dropout=dropout)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x


class Transformer(nn.Module):
    """Vision Transformer Encoder"""

    def __init__(self, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4., dropout=0.1):
        super().__init__()
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        return x


class DecoderBlock(nn.Module):
    """Decoder block with skip connections"""

    def __init__(self, in_channels, out_channels, skip_channels=0):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels + skip_channels, out_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, skip=None):
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        return x


class TransUNet(nn.Module):
    """TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation"""

    def __init__(self, img_size=224, patch_size=16, in_channels=3, out_channels=1,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4., dropout=0.1):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.embed_dim = embed_dim

        # CNN encoder (first few layers)
        self.encoder1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

        self.encoder3 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )

        # Patch embedding
        self.patch_embed = PatchEmbedding(img_size // 4, patch_size, 256, embed_dim)

        # Positional embedding
        self.pos_embed = nn.Parameter(torch.zeros(1, self.patch_embed.n_patches, embed_dim))

        # Transformer encoder
        self.transformer = Transformer(embed_dim, depth, num_heads, mlp_ratio, dropout)

        # Decoder
        self.decoder0 = nn.Sequential(
            nn.Conv2d(embed_dim, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )

        self.decoder1 = DecoderBlock(256, 256, skip_channels=256)
        self.decoder2 = DecoderBlock(256, 128, skip_channels=128)
        self.decoder3 = DecoderBlock(128, 64, skip_channels=64)

        # Final output
        self.out_conv = nn.Conv2d(64, out_channels, 1)

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        """Initialize weights for better convergence"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Encoder
        skip1 = self.encoder1(x)  # 224x224x64
        skip2 = self.encoder2(skip1)  # 112x112x128
        skip3 = self.encoder3(skip2)  # 56x56x256

        # Patch embedding
        x = self.patch_embed(skip3)  # (B, n_patches, embed_dim)

        # Add positional embedding
        x = x + self.pos_embed

        # Transformer
        x = self.transformer(x)

        # Reshape for decoder
        B, n_patches, embed_dim = x.shape
        h = w = int(np.sqrt(n_patches))
        x = x.transpose(1, 2).reshape(B, embed_dim, h, w)

        # Decoder
        x = self.decoder0(x)  # Output: 256 channels, h x w spatial dims

        # Upsample to match skip3 size (56x56)
        x = F.interpolate(x, size=skip3.shape[2:], mode='bilinear', align_corners=True)
        x = self.decoder1(x, skip3)  # 56x56x256

        # Upsample to match skip2 size (112x112)
        x = F.interpolate(x, size=skip2.shape[2:], mode='bilinear', align_corners=True)
        x = self.decoder2(x, skip2)  # 112x112x128

        # Upsample to match skip1 size (224x224)
        x = F.interpolate(x, size=skip1.shape[2:], mode='bilinear', align_corners=True)
        x = self.decoder3(x, skip1)  # 224x224x64

        # Output
        x = self.out_conv(x)
        x = torch.sigmoid(x)

        return x


def get_transunet(img_size=224, in_channels=3, out_channels=1, pretrained=True):
    """Create TransUNet model with optional pretrained weights

    """

    model = TransUNet(
        img_size=img_size,
        patch_size=16,
        in_channels=in_channels,
        out_channels=out_channels,
        embed_dim=768,
        depth=12,
        num_heads=12,
        mlp_ratio=4.,
        dropout=0.1
    )

    if pretrained:
        print("Loading pretrained ViT weights...")
        try:
            # Load pretrained ViT-B/16 weights from timm
            import timm
            vit_model = timm.create_model('vit_base_patch16_224', pretrained=True)

            # Transfer weights from ViT to TransUNet transformer
            model_dict = model.state_dict()
            pretrained_dict = {}

            # Map ViT weights to TransUNet
            for i in range(12):  # 12 transformer blocks
                # Attention weights
                pretrained_dict[f'transformer.blocks.{i}.attn.qkv.weight'] = \
                    vit_model.blocks[i].attn.qkv.weight
                pretrained_dict[f'transformer.blocks.{i}.attn.qkv.bias'] = \
                    vit_model.blocks[i].attn.qkv.bias
                pretrained_dict[f'transformer.blocks.{i}.attn.proj.weight'] = \
                    vit_model.blocks[i].attn.proj.weight
                pretrained_dict[f'transformer.blocks.{i}.attn.proj.bias'] = \
                    vit_model.blocks[i].attn.proj.bias

                # MLP weights
                pretrained_dict[f'transformer.blocks.{i}.mlp.fc1.weight'] = \
                    vit_model.blocks[i].mlp.fc1.weight
                pretrained_dict[f'transformer.blocks.{i}.mlp.fc1.bias'] = \
                    vit_model.blocks[i].mlp.fc1.bias
                pretrained_dict[f'transformer.blocks.{i}.mlp.fc2.weight'] = \
                    vit_model.blocks[i].mlp.fc2.weight
                pretrained_dict[f'transformer.blocks.{i}.mlp.fc2.bias'] = \
                    vit_model.blocks[i].mlp.fc2.bias

                # Layer norms
                pretrained_dict[f'transformer.blocks.{i}.norm1.weight'] = \
                    vit_model.blocks[i].norm1.weight
                pretrained_dict[f'transformer.blocks.{i}.norm1.bias'] = \
                    vit_model.blocks[i].norm1.bias
                pretrained_dict[f'transformer.blocks.{i}.norm2.weight'] = \
                    vit_model.blocks[i].norm2.weight
                pretrained_dict[f'transformer.blocks.{i}.norm2.bias'] = \
                    vit_model.blocks[i].norm2.bias

            # Final norm
            pretrained_dict['transformer.norm.weight'] = vit_model.norm.weight
            pretrained_dict['transformer.norm.bias'] = vit_model.norm.bias

            # Update model with pretrained weights
            model_dict.update(pretrained_dict)
            model.load_state_dict(model_dict, strict=False)

            print("✓ Successfully loaded pretrained ViT weights!")
            print(f"  Loaded {len(pretrained_dict)} pretrained parameters")

        except Exception as e:
            print(f"⚠ Warning: Could not load pretrained weights: {str(e)}")
            print("  Continuing with random initialization...")

    return model

In [6]:
def download_pretrained_transunet():
    """
    Download pretrained TransUNet weights

    Official weights from: https://github.com/Beckschen/TransUNet
    """

    import gdown

    # Create cache directory
    cache_dir = os.path.expanduser('~/.cache/transunet')
    os.makedirs(cache_dir, exist_ok=True)

    weights_path = os.path.join(cache_dir, 'transunet_synapse.pth')

    if os.path.exists(weights_path):
        print(f"✓ Using cached TransUNet weights: {weights_path}")
        return weights_path

        try:
            from huggingface_hub import hf_hub_download
            weights_path = hf_hub_download(
                repo_id="Beckschen/TransUNet",
                filename="transunet_pretrained.pth",
                cache_dir=cache_dir
            )
            print(f"✓ Downloaded from HuggingFace: {weights_path}")
            return weights_path
        except:
            print("⚠ Could not download from HuggingFace either")
            return None


def load_pretrained_transunet(model, strict=False):
    """
    Load pretrained TransUNet weights into model

    """

    weights_path = download_pretrained_transunet()

    if weights_path is None:
        print("⚠ Could not load pretrained TransUNet weights")
        print("  Falling back to ImageNet ViT weights only")
        return model


        # Load checkpoint
        checkpoint = torch.load(weights_path, map_location='cpu')

        # Extract state dict
        if isinstance(checkpoint, dict) and 'model' in checkpoint:
            state_dict = checkpoint['model']
        elif isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']
        else:
            state_dict = checkpoint

        # Load weights
        model_dict = model.state_dict()

        # Filter out mismatched keys (output layer might be different size)
        pretrained_dict = {}
        skipped_keys = []

        for k, v in state_dict.items():
            # Remove 'module.' prefix if present
            key = k.replace('module.', '')

            if key in model_dict:
                if v.shape == model_dict[key].shape:
                    pretrained_dict[key] = v
                else:
                    skipped_keys.append(f"{key} (shape mismatch: {v.shape} vs {model_dict[key].shape})")
            else:
                skipped_keys.append(f"{key} (not in model)")

        # Update model
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict, strict=strict)

        print(f"✓ Loaded pretrained TransUNet weights!")
        print(f"  Loaded {len(pretrained_dict)} parameters")

        if skipped_keys:
            print(f"  Skipped {len(skipped_keys)} parameters:")
            for key in skipped_keys[:5]:  # Show first 5
                print(f"    - {key}")
            if len(skipped_keys) > 5:
                print(f"    ... and {len(skipped_keys) - 5} more")

        return model



def load_vit_weights_only(model):
    """
    Load only ViT (transformer) weights from timm
    This is a fallback when full TransUNet weights are not available
    """

    import timm


    print("Loading ImageNet pretrained ViT weights...")
    vit_model = timm.create_model('vit_base_patch16_224', pretrained=True)

    model_dict = model.state_dict()
    pretrained_dict = {}

    # Map ViT weights to TransUNet transformer
    for i in range(12):  # 12 transformer blocks
        # Attention weights
        for param in ['qkv.weight', 'qkv.bias', 'proj.weight', 'proj.bias']:
            src_key = f'blocks.{i}.attn.{param}'
            dst_key = f'transformer.blocks.{i}.attn.{param}'
            if hasattr(vit_model, 'blocks') and src_key.split('.')[0] in dir(vit_model):
                pretrained_dict[dst_key] = getattr(vit_model.blocks[i].attn, param.split('.')[0])

        # MLP weights
        for param in ['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias']:
            src_key = f'blocks.{i}.mlp.{param}'
            dst_key = f'transformer.blocks.{i}.mlp.{param}'
            if hasattr(vit_model, 'blocks'):
                pretrained_dict[dst_key] = getattr(vit_model.blocks[i].mlp, param.split('.')[0])

        # Layer norms
        for norm_idx in ['1', '2']:
            for param in ['weight', 'bias']:
                src_key = f'blocks.{i}.norm{norm_idx}.{param}'
                dst_key = f'transformer.blocks.{i}.norm{norm_idx}.{param}'
                if hasattr(vit_model, 'blocks'):
                    pretrained_dict[dst_key] = getattr(vit_model.blocks[i], f'norm{norm_idx}')

    # Final norm
    if hasattr(vit_model, 'norm'):
        pretrained_dict['transformer.norm.weight'] = vit_model.norm.weight
        pretrained_dict['transformer.norm.bias'] = vit_model.norm.bias

    # Update model
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict, strict=False)

    print(f"✓ Loaded ImageNet ViT weights")
    print(f"  Loaded {len(pretrained_dict)} parameters")

    return model


def load_from_google_drive_direct(model):
    """
    Direct download of TransUNet weights with multiple fallback options
    """

    import requests

    # Multiple potential sources
    sources = [
        {
            'name': 'TransUNet Synapse (Official)',
            'url': 'https://drive.google.com/uc?export=download&id=1kfP_1Fo4r0PHW1DXuKLKPHLLP0y1LXJ5',
            'type': 'google_drive'
        },
        {
            'name': 'TransUNet R50-ViT-B (Alternative)',
            'url': 'https://storage.googleapis.com/vit_models/imagenet21k/R50+ViT-B_16.npz',
            'type': 'numpy'
        }
    ]

    for source in sources:

          print(f"\nTrying {source['name']}...")

          if source['type'] == 'google_drive':
              import gdown
              cache_dir = os.path.expanduser('~/.cache/transunet')
              os.makedirs(cache_dir, exist_ok=True)
              weights_path = os.path.join(cache_dir, 'pretrained.pth')

              gdown.download(source['url'], weights_path, quiet=False)
              checkpoint = torch.load(weights_path, map_location='cpu')

              # Load into model
              if isinstance(checkpoint, dict) and 'model' in checkpoint:
                  model.load_state_dict(checkpoint['model'], strict=False)
              else:
                  model.load_state_dict(checkpoint, strict=False)

              print(f"Successfully loaded from {source['name']}")
              return model


    return load_vit_weights_only(model)

In [7]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix


def calculate_iou(pred, target, threshold=0.5):
    """Calculate IoU (Intersection over Union) for binary segmentation"""

    pred = (pred > threshold).float()
    target = target.float()

    # Flatten
    pred = pred.view(-1)
    target = target.view(-1)

    # Calculate intersection and union
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection

    # Avoid division by zero
    if union == 0:
        return 1.0 if intersection == 0 else 0.0

    iou = intersection / union
    return iou.item()


def calculate_miou(pred, target, threshold=0.5):
    """Calculate mean IoU (same as IoU for binary segmentation)"""
    return calculate_iou(pred, target, threshold)


def calculate_dice(pred, target, threshold=0.5, smooth=1.0):
    """Calculate Dice coefficient"""

    pred = (pred > threshold).float()
    target = target.float()

    # Flatten
    pred = pred.view(-1)
    target = target.view(-1)

    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

    return dice.item()


def calculate_pixel_accuracy(pred, target, threshold=0.5):
    """Calculate pixel-wise accuracy"""

    pred = (pred > threshold).float()
    target = target.float()

    correct = (pred == target).float().sum()
    total = target.numel()

    accuracy = correct / total
    return accuracy.item()


def calculate_precision_recall(pred, target, threshold=0.5):
    """Calculate precision and recall"""

    pred = (pred > threshold).float()
    target = target.float()

    # Flatten
    pred = pred.view(-1)
    target = target.view(-1)

    # True positives, false positives, false negatives
    tp = (pred * target).sum()
    fp = (pred * (1 - target)).sum()
    fn = ((1 - pred) * target).sum()

    # Calculate precision and recall
    precision = tp / (tp + fp + 1e-7)
    recall = tp / (tp + fn + 1e-7)

    return precision.item(), recall.item()


def calculate_f1_score(pred, target, threshold=0.5):
    """Calculate F1 score"""

    precision, recall = calculate_precision_recall(pred, target, threshold)

    if precision + recall == 0:
        return 0.0

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1


class MetricsTracker:
    """Track and compute metrics during training/evaluation"""

    def __init__(self):
        self.reset()

    def reset(self):
        """Reset all metrics"""
        self.iou_scores = []
        self.dice_scores = []
        self.pixel_acc_scores = []
        self.precision_scores = []
        self.recall_scores = []
        self.f1_scores = []

    def update(self, pred, target, threshold=0.5):
        """Update metrics with new predictions"""

        # Calculate all metrics
        iou = calculate_iou(pred, target, threshold)
        dice = calculate_dice(pred, target, threshold)
        pixel_acc = calculate_pixel_accuracy(pred, target, threshold)
        precision, recall = calculate_precision_recall(pred, target, threshold)
        f1 = calculate_f1_score(pred, target, threshold)

        # Store metrics
        self.iou_scores.append(iou)
        self.dice_scores.append(dice)
        self.pixel_acc_scores.append(pixel_acc)
        self.precision_scores.append(precision)
        self.recall_scores.append(recall)
        self.f1_scores.append(f1)

    def get_metrics(self):
        """Get average metrics"""

        metrics = {
            'IoU': np.mean(self.iou_scores) if self.iou_scores else 0.0,
            'mIoU': np.mean(self.iou_scores) if self.iou_scores else 0.0,  # Same as IoU for binary
            'Dice': np.mean(self.dice_scores) if self.dice_scores else 0.0,
            'Pixel_Acc': np.mean(self.pixel_acc_scores) if self.pixel_acc_scores else 0.0,
            'Precision': np.mean(self.precision_scores) if self.precision_scores else 0.0,
            'Recall': np.mean(self.recall_scores) if self.recall_scores else 0.0,
            'F1': np.mean(self.f1_scores) if self.f1_scores else 0.0
        }

        return metrics

    def print_metrics(self, prefix=''):
        """Print current metrics"""

        metrics = self.get_metrics()

        print(f"\n{prefix} Metrics:")
        print(f"  IoU/mIoU: {metrics['mIoU']:.4f}")
        print(f"  Dice:     {metrics['Dice']:.4f}")
        print(f"  Pixel Acc: {metrics['Pixel_Acc']:.4f}")
        print(f"  Precision: {metrics['Precision']:.4f}")
        print(f"  Recall:    {metrics['Recall']:.4f}")
        print(f"  F1 Score:  {metrics['F1']:.4f}")

        return metrics

In [ ]:
class Trainer:
    """Main training class for TransUNet"""

    def __init__(self, config):
        self.config = config
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        # Create save directory
        self.save_dir = os.path.join(config['save_dir'],
                                     f"{config['loss_function']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
        os.makedirs(self.save_dir, exist_ok=True)

        # Save config
        with open(os.path.join(self.save_dir, 'config.json'), 'w') as f:
            json.dump(config, f, indent=4)

        # Initialize model
        self.model = get_transunet(
            img_size=config['img_size'],
            in_channels=3,
            out_channels=1,
            pretrained=config.get('pretrained', True)  # Use pretrained by default
        ).to(self.device)

        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()) / 1e6:.2f}M")

        # Initialize loss function
        self.criterion = get_loss_function(config['loss_function'])
        print(f"Using loss function: {config['loss_function']}")

        # Initialize optimizer
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=config['learning_rate'],
            weight_decay=config['weight_decay']
        )

        # Initialize scheduler
        if config['scheduler'] == 'reduce_on_plateau':
            self.scheduler = ReduceLROnPlateau(
                self.optimizer, mode='max', factor=0.5, patience=5
            )
        elif config['scheduler'] == 'cosine':
            self.scheduler = CosineAnnealingLR(
                self.optimizer, T_max=config['epochs'], eta_min=1e-6
            )
        else:
            self.scheduler = None

        # Initialize data loaders
        self.train_loader, self.val_loader, self.test_loader = get_dataloaders(
            root_dir=config['data_dir'],
            img_size=config['img_size'],
            batch_size=config['batch_size'],
            num_workers=config['num_workers']
        )

        # Training history
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'train_miou': [],
            'val_miou': [],
            'learning_rate': []
        }

        self.best_val_miou = 0.0
        self.best_epoch = 0

    def train_epoch(self, epoch):
        """Train for one epoch"""

        self.model.train()
        metrics_tracker = MetricsTracker()
        total_loss = 0.0

        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config['epochs']} [Train]")

        for batch_idx, (images, masks) in enumerate(pbar):
            images = images.to(self.device)
            masks = masks.to(self.device)

            # Forward pass
            self.optimizer.zero_grad()
            outputs = self.model(images)

            # Calculate loss
            loss = self.criterion(outputs, masks)

            # Backward pass
            loss.backward()

            # Gradient clipping
            if self.config.get('grad_clip', 0) > 0:
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config['grad_clip'])

            self.optimizer.step()

            # Update metrics
            total_loss += loss.item()
            metrics_tracker.update(outputs.detach(), masks.detach())

            # Update progress bar
            pbar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'avg_loss': f"{total_loss / (batch_idx + 1):.4f}"
            })

        # Calculate average metrics
        avg_loss = total_loss / len(self.train_loader)
        metrics = metrics_tracker.get_metrics()

        return avg_loss, metrics

    def validate(self, epoch):
        """Validate the model"""

        self.model.eval()
        metrics_tracker = MetricsTracker()
        total_loss = 0.0

        with torch.no_grad():
            pbar = tqdm(self.val_loader, desc=f"Epoch {epoch+1}/{self.config['epochs']} [Val]")

            for images, masks in pbar:
                images = images.to(self.device)
                masks = masks.to(self.device)

                # Forward pass
                outputs = self.model(images)

                # Calculate loss
                loss = self.criterion(outputs, masks)

                # Update metrics
                total_loss += loss.item()
                metrics_tracker.update(outputs, masks)

                # Update progress bar
                pbar.set_postfix({'loss': f"{loss.item():.4f}"})

        # Calculate average metrics
        avg_loss = total_loss / len(self.val_loader)
        metrics = metrics_tracker.get_metrics()

        return avg_loss, metrics

    def test(self):
        """Test the model on test set"""

        print("\n" + "="*50)
        print("Testing on Test Set")
        print("="*50)

        # Load best model
        checkpoint_path = os.path.join(self.save_dir, 'best_model.pth')
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, weights_only=False)  # Fixed for PyTorch 2.6
            self.model.load_state_dict(checkpoint['model_state_dict'])
            print(f"Loaded best model from epoch {checkpoint['epoch']}")

        self.model.eval()
        metrics_tracker = MetricsTracker()
        total_loss = 0.0

        with torch.no_grad():
            pbar = tqdm(self.test_loader, desc="Testing")

            for images, masks in pbar:
                images = images.to(self.device)
                masks = masks.to(self.device)

                # Forward pass
                outputs = self.model(images)

                # Calculate loss
                loss = self.criterion(outputs, masks)

                # Update metrics
                total_loss += loss.item()
                metrics_tracker.update(outputs, masks)

        # Calculate and print metrics
        avg_loss = total_loss / len(self.test_loader)
        print(f"\nTest Loss: {avg_loss:.4f}")
        test_metrics = metrics_tracker.print_metrics(prefix="Test")

        # Save test results
        test_results = {
            'test_loss': avg_loss,
            'test_metrics': test_metrics,
            'loss_function': self.config['loss_function']
        }

        with open(os.path.join(self.save_dir, 'test_results.json'), 'w') as f:
            json.dump(test_results, f, indent=4)

        return test_metrics

    def save_checkpoint(self, epoch, val_miou, is_best=False):
        """Save model checkpoint"""

        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'val_miou': val_miou,
            'config': self.config,
            'history': self.history
        }

        # Save latest checkpoint
        checkpoint_path = os.path.join(self.save_dir, 'latest_model.pth')
        torch.save(checkpoint, checkpoint_path)

        # Save best checkpoint
        if is_best:
            best_path = os.path.join(self.save_dir, 'best_model.pth')
            torch.save(checkpoint, best_path)
            print(f"✓ Best model saved with mIoU: {val_miou:.4f}")

    def plot_history(self):
        """Plot training history"""

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Plot loss
        axes[0, 0].plot(self.history['train_loss'], label='Train Loss')
        axes[0, 0].plot(self.history['val_loss'], label='Val Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].set_title(f'Loss Curves ({self.config["loss_function"]})')
        axes[0, 0].legend()
        axes[0, 0].grid(True)

        # Plot mIoU
        axes[0, 1].plot(self.history['train_miou'], label='Train mIoU')
        axes[0, 1].plot(self.history['val_miou'], label='Val mIoU')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('mIoU')
        axes[0, 1].set_title('mIoU Curves')
        axes[0, 1].legend()
        axes[0, 1].grid(True)

        # Plot learning rate
        axes[1, 0].plot(self.history['learning_rate'])
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Learning Rate')
        axes[1, 0].set_title('Learning Rate Schedule')
        axes[1, 0].grid(True)

        # Plot best metrics
        axes[1, 1].text(0.5, 0.5,
                       f"Best Val mIoU: {self.best_val_miou:.4f}\n"
                       f"Best Epoch: {self.best_epoch}\n"
                       f"Loss Function: {self.config['loss_function']}",
                       ha='center', va='center', fontsize=14)
        axes[1, 1].axis('off')

        plt.tight_layout()
        plt.savefig(os.path.join(self.save_dir, 'training_history.png'), dpi=300, bbox_inches='tight')
        plt.close()

    def train(self):
        """Main training loop"""

        print("\n" + "="*50)
        print(f"Starting Training with {self.config['loss_function']} Loss")
        print("="*50 + "\n")

        for epoch in range(self.config['epochs']):
            # Train
            train_loss, train_metrics = self.train_epoch(epoch)

            # Validate
            val_loss, val_metrics = self.validate(epoch)

            # Update history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_miou'].append(train_metrics['mIoU'])
            self.history['val_miou'].append(val_metrics['mIoU'])
            self.history['learning_rate'].append(self.optimizer.param_groups[0]['lr'])

            # Print epoch summary
            print(f"\nEpoch {epoch+1}/{self.config['epochs']}:")
            print(f"  Train Loss: {train_loss:.4f} | Train mIoU: {train_metrics['mIoU']:.4f}")
            print(f"  Val Loss:   {val_loss:.4f} | Val mIoU:   {val_metrics['mIoU']:.4f}")
            print(f"  LR: {self.optimizer.param_groups[0]['lr']:.6f}")

            # Update learning rate
            if self.scheduler is not None:
                if isinstance(self.scheduler, ReduceLROnPlateau):
                    self.scheduler.step(val_metrics['mIoU'])
                else:
                    self.scheduler.step()

            # Check if best model
            is_best = val_metrics['mIoU'] > self.best_val_miou
            if is_best:
                self.best_val_miou = val_metrics['mIoU']
                self.best_epoch = epoch + 1

            # Save checkpoint
            self.save_checkpoint(epoch, val_metrics['mIoU'], is_best)

            # Plot history
            self.plot_history()

        print("\n" + "="*50)
        print("Training Complete!")
        print(f"Best Val mIoU: {self.best_val_miou:.4f} at Epoch {self.best_epoch}")
        print("="*50 + "\n")

        # Test on test set
        test_metrics = self.test()

        return self.history, test_metrics


def main():
    """Main function to run experiments with different loss functions"""

    # Base configuration
    base_config = {
        'data_dir': '/content/plantsegv2',  # Update this path
        'save_dir': '/content/drive/MyDrive/ELEC_576/Final_Project/experiments',
        'img_size': 224,
        'batch_size': 16,
        'num_workers': 4,
        'epochs': 50,
        'learning_rate': 1e-4,
        'weight_decay': 1e-5,
        'grad_clip': 1.0,
        'scheduler': 'reduce_on_plateau',
        'pretrained': True  # Use pretrained ViT weights
    }

    # Loss functions to experiment with
    loss_functions = ['bce', 'dice', 'focal', 'bce_dice']

    # Results storage
    all_results = {}

    # Train with each loss function
    for loss_fn in loss_functions:
        print("\n" + "="*70)
        print(f"Training with {loss_fn.upper()} Loss Function")
        print("="*70)

        # Create config for this experiment
        config = base_config.copy()
        config['loss_function'] = loss_fn

        # Train model
        trainer = Trainer(config)
        history, test_metrics = trainer.train()

        # Store results
        all_results[loss_fn] = {
            'best_val_miou': trainer.best_val_miou,
            'best_epoch': trainer.best_epoch,
            'test_metrics': test_metrics,
            'save_dir': trainer.save_dir
        }

    # Compare results
    print("\n" + "="*70)
    print("FINAL COMPARISON OF LOSS FUNCTIONS")
    print("="*70)

    comparison_table = []
    for loss_fn, results in all_results.items():
        comparison_table.append({
            'Loss Function': loss_fn.upper(),
            'Best Val mIoU': f"{results['best_val_miou']:.4f}",
            'Test mIoU': f"{results['test_metrics']['mIoU']:.4f}",
            'Test Dice': f"{results['test_metrics']['Dice']:.4f}",
            'Test F1': f"{results['test_metrics']['F1']:.4f}"
        })

    # Print comparison table
    print("\n{:<15} {:<15} {:<15} {:<15} {:<15}".format(
        "Loss Function", "Best Val mIoU", "Test mIoU", "Test Dice", "Test F1"))
    print("-" * 75)
    for row in comparison_table:
        print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(
            row['Loss Function'], row['Best Val mIoU'], row['Test mIoU'],
            row['Test Dice'], row['Test F1']))

    # Save comparison results
    with open(os.path.join(base_config['save_dir'], 'comparison_results.json'), 'w') as f:
        json.dump(all_results, f, indent=4)

    # Plot comparison
    plot_comparison(all_results, base_config['save_dir'])

    print(f"Results saved in: {base_config['save_dir']}")


def plot_comparison(all_results, save_dir):
    """Plot comparison of different loss functions"""

    loss_names = list(all_results.keys())
    val_mious = [all_results[ln]['best_val_miou'] for ln in loss_names]
    test_mious = [all_results[ln]['test_metrics']['mIoU'] for ln in loss_names]
    test_dice = [all_results[ln]['test_metrics']['Dice'] for ln in loss_names]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Val mIoU comparison
    axes[0].bar(loss_names, val_mious, color='skyblue', edgecolor='navy')
    axes[0].set_ylabel('mIoU')
    axes[0].set_title('Best Validation mIoU by Loss Function')
    axes[0].set_ylim([0, 1])
    axes[0].grid(axis='y', alpha=0.3)

    # Test mIoU comparison
    axes[1].bar(loss_names, test_mious, color='lightcoral', edgecolor='darkred')
    axes[1].set_ylabel('mIoU')
    axes[1].set_title('Test mIoU by Loss Function')
    axes[1].set_ylim([0, 1])
    axes[1].grid(axis='y', alpha=0.3)

    # Test Dice comparison
    axes[2].bar(loss_names, test_dice, color='lightgreen', edgecolor='darkgreen')
    axes[2].set_ylabel('Dice Score')
    axes[2].set_title('Test Dice Score by Loss Function')
    axes[2].set_ylim([0, 1])
    axes[2].grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'loss_function_comparison.png'), dpi=300, bbox_inches='tight')
    plt.close()


if __name__ == '__main__':
    main()


Training with BCE Loss Function
Using device: cuda
Loading pretrained ViT weights...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

✓ Successfully loaded pretrained ViT weights!
  Loaded 146 pretrained parameters


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


...


Epoch 49/50:
  Train Loss: 0.2585 | Train mIoU: 0.6261
  Val Loss:   0.3095 | Val mIoU:   0.5585
  LR: 0.000050


Epoch 50/50 [Val]: 100%|██████████| 78/78 [00:10<00:00,  7.14it/s, loss=0.2604]



Epoch 50/50:
  Train Loss: 0.2568 | Train mIoU: 0.6269
  Val Loss:   0.3094 | Val mIoU:   0.5593
  LR: 0.000050

Training Complete!
Best Val mIoU: 0.5632 at Epoch 48


Testing on Test Set
Loaded best model from epoch 47


Testing: 100%|██████████| 144/144 [00:22<00:00,  6.47it/s]



Test Loss: 0.3152

Test Metrics:
  IoU/mIoU: 0.5565
  Dice:     0.7046
  Pixel Acc: 0.8884
  Precision: 0.7065
  Recall:    0.7289
  F1 Score:  0.7046

FINAL COMPARISON OF LOSS FUNCTIONS

Loss Function   Best Val mIoU   Test mIoU       Test Dice       Test F1        
---------------------------------------------------------------------------
BCE             0.5453          0.5484          0.6967          0.6967         
DICE            0.5589          0.5498          0.6991          0.6991         
FOCAL           0.5380          0.5339          0.6852          0.6852         
BCE_DICE        0.5632          0.5565          0.7046          0.7046         

✓ All experiments completed!
Results saved in: /content/drive/MyDrive/ELEC_576/Final_Project/experiments


In [11]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Rectangle
import pandas as pd

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'

# results (Test metrics only)
results = {
    'SegNeXt\n(Baseline)': {
        'Test mIoU': 0.4452,
        'Test mAcc': 0.5995,
    },
    'BCE\n(Ours)': {
        'Test mIoU': 0.5484,
        'Test mAcc': 0.8843,
    },
    'Dice\n(Ours)': {
        'Test mIoU': 0.5498,
        'Test mAcc': 0.8783,
    },
    'Focal\n(Ours)': {
        'Test mIoU': 0.5339,
        'Test mAcc': 0.8881,
    },
    'BCE-Dice\n(Ours)': {
        'Test mIoU': 0.5565,
        'Test mAcc': 0.8884,
    }
}

# Create DataFrame
df = pd.DataFrame(results).T



def plot_comprehensive_comparison():
    """Create side-by-side bar chart comparing all metrics"""

    fig, axes = plt.subplots(2, 1, figsize=(16, 10))
    fig.suptitle('TransUNet vs SegNeXt Baseline on PlantSeg Dataset\nTest Set Performance Comparison',
                 fontsize=16, fontweight='bold', y=0.995)

    metrics = ['Test mIoU', 'Test mAcc']
    titles = ['Test mIoU (Mean Intersection over Union)', 'Test mAcc (Mean Accuracy)']
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

    loss_names = list(results.keys())

    for idx, (ax, metric, title, color) in enumerate(zip(axes.flat, metrics, titles, colors)):
        values = [results[loss][metric] for loss in loss_names]
        x_pos = np.arange(len(loss_names))

        # Different color for baseline
        bar_colors = ['#808080'] + [color] * 4  # Gray for baseline, color for ours
        bars = ax.bar(x_pos, values, color=bar_colors, alpha=0.8, edgecolor='black', linewidth=1.5)

        # Add value labels on bars
        for bar, val in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{val:.4f}',
                   ha='center', va='bottom', fontsize=9, fontweight='bold')

        # Highlight best performer among our methods
        best_idx = values.index(max(values))
        bars[best_idx].set_edgecolor('gold')
        bars[best_idx].set_linewidth(3)

        ax.set_ylabel(metric.replace('Test ', ''), fontsize=11, fontweight='bold')
        ax.set_title(title, fontsize=12, fontweight='bold', pad=10)
        ax.set_ylim([min(values) * 0.92, max(values) * 1.08])
        ax.set_xticks(x_pos)
        ax.set_xticklabels(loss_names, fontsize=9, rotation=15, ha='right')
        ax.grid(axis='y', alpha=0.3, linestyle='--')
        ax.set_axisbelow(True)

        # Add horizontal line at baseline
        ax.axhline(y=values[0], color='gray', linestyle='--', linewidth=2, alpha=0.5, label='Baseline')

        # Add improvement text for best method
        if best_idx > 0:
            improvement = ((values[best_idx] - values[0]) / values[0]) * 100
            ax.text(0.98, 0.98, f'Best: +{improvement:.1f}%\nvs baseline',
                   transform=ax.transAxes, fontsize=8, fontweight='bold',
                   ha='right', va='top',
                   bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

    plt.tight_layout()
    plt.savefig('loss_comparison_comprehensive.png', dpi=300, bbox_inches='tight')
    plt.close()


if __name__ == '__main__':
    plot_comprehensive_comparison()

In [20]:
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import glob


class LandscapeComparisonGenerator:
    """Generate landscape comparison figures for paper"""

    def __init__(self, checkpoint_paths, img_size=224, device=None):
        self.img_size = img_size
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.models = {}
        self.model_info = {}

        for loss_name, checkpoint_path in checkpoint_paths.items():
            print(f"\nLoading {loss_name} model...")
            model = get_transunet(img_size=img_size, in_channels=3, out_channels=1)
            checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
            model.load_state_dict(checkpoint['model_state_dict'])
            model.to(self.device)
            model.eval()

            self.models[loss_name] = model
            self.model_info[loss_name] = {
                'epoch': checkpoint['epoch'],
                'val_miou': checkpoint.get('val_miou', 0),
                'path': checkpoint_path
            }
            print(f"Loaded from epoch {checkpoint['epoch']}, Val mIoU: {checkpoint.get('val_miou', 0):.4f}")

        print(f"\nAll {len(self.models)} models loaded successfully!")

        self.transform = A.Compose([
            A.Resize(img_size, img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

    def predict_all(self, image_path, threshold=0.5):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_size = image.shape[:2]

        transformed = self.transform(image=image)
        image_tensor = transformed['image'].unsqueeze(0).to(self.device)

        results = {}
        with torch.no_grad():
            for loss_name, model in self.models.items():
                output = model(image_tensor)
                prob_mask = torch.sigmoid(output).squeeze().cpu().numpy()
                mask = (prob_mask > threshold).astype(np.uint8)
                prob_mask_resized = cv2.resize(prob_mask, (original_size[1], original_size[0]))
                mask_resized = cv2.resize(mask, (original_size[1], original_size[0]))
                results[loss_name] = (mask_resized, prob_mask_resized)

        return results, image

    def create_simple_plantseg_example(self, image_path, save_path, ground_truth_path, threshold=0.5):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_size = image.shape[:2]

        transformed = self.transform(image=image)
        image_tensor = transformed['image'].unsqueeze(0).to(self.device)

        results, image = self.predict_all(image_path, threshold)

        ground_truth = cv2.imread(ground_truth_path, cv2.IMREAD_GRAYSCALE)
        if ground_truth.max() > 1:
            ground_truth = (ground_truth / 255.0)

        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        axes[0].imshow(image)
        axes[0].set_title('RGB Leaf Image', fontsize=14, fontweight='bold')
        axes[0].axis('off')

        axes[1].imshow(ground_truth, cmap='gray', vmin=0, vmax=1)
        axes[1].set_title('Ground Truth\nDiseased Pixels', fontsize=14, fontweight='bold')
        axes[1].axis('off')

        mask, prob_mask = results['BCE_DICE']
        axes[2].imshow(prob_mask, cmap='hot', vmin=0, vmax=1)
        axes[2].set_title('Prediction\nViT+BCE+Dice Model', fontsize=14, fontweight='bold')
        axes[2].axis('off')

        img_name = os.path.basename(image_path)
        fig.suptitle(f'PlantSeg Example - {img_name}', fontsize=16, fontweight='bold', y=0.98)

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"Simple comparison saved to: {save_path}")
        plt.close()

    def create_landscape_comparison(self, image_path, save_path, ground_truth_path=None,
                                   threshold=0.5, show_probability=True):
        results, image = self.predict_all(image_path, threshold)

        ground_truth = None
        if ground_truth_path and os.path.exists(ground_truth_path):
            ground_truth = cv2.imread(ground_truth_path, cv2.IMREAD_GRAYSCALE)
            if ground_truth.max() > 1:
                ground_truth = (ground_truth / 255.0)

        fig, axes = plt.subplots(2, 3, figsize=(18, 12))

        axes[0, 0].imshow(image)
        axes[0, 0].set_title('Original Image', fontsize=16, fontweight='bold', pad=10)
        axes[0, 0].axis('off')

        if ground_truth is not None:
            axes[0, 1].imshow(ground_truth, cmap='gray', vmin=0, vmax=1)
            axes[0, 1].set_title('Ground Truth Mask', fontsize=16, fontweight='bold', pad=10)
        else:
            axes[0, 1].text(0.5, 0.5, 'No Ground Truth',
                          transform=axes[0, 1].transAxes,
                          ha='center', va='center', fontsize=14)
        axes[0, 1].axis('off')

        mask, prob_mask = results['BCE']
        if show_probability:
            im = axes[0, 2].imshow(prob_mask, cmap='hot', vmin=0, vmax=1)
            axes[0, 2].set_title('BCE\nProbability Map', fontsize=14, fontweight='bold', pad=10)
        else:
            axes[0, 2].imshow(mask, cmap='gray', vmin=0, vmax=1)
            axes[0, 2].set_title('BCE\nSmooth, clean masks', fontsize=14, fontweight='bold', pad=10)
        axes[0, 2].axis('off')

        mask, prob_mask = results['DICE']
        if show_probability:
            axes[1, 0].imshow(prob_mask, cmap='hot', vmin=0, vmax=1)
            axes[1, 0].set_title('Dice\nProbability Map', fontsize=14, fontweight='bold', pad=10)
        else:
            axes[1, 0].imshow(mask, cmap='gray', vmin=0, vmax=1)
            axes[1, 0].set_title('Dice\nFull, minor false+', fontsize=14, fontweight='bold', pad=10)
        axes[1, 0].axis('off')

        mask, prob_mask = results['FOCAL']
        if show_probability:
            axes[1, 1].imshow(prob_mask, cmap='hot', vmin=0, vmax=1)
            axes[1, 1].set_title('Focal\nProbability Map', fontsize=14, fontweight='bold', pad=10)
        else:
            axes[1, 1].imshow(mask, cmap='gray', vmin=0, vmax=1)
            axes[1, 1].set_title('Focal\nConcentrated, misses edges', fontsize=14, fontweight='bold', pad=10)
        axes[1, 1].axis('off')

        mask, prob_mask = results['BCE_DICE']
        if show_probability:
            axes[1, 2].imshow(prob_mask, cmap='hot', vmin=0, vmax=1)
            axes[1, 2].set_title('BCE+Dice ★\nProbability Map', fontsize=14,
                               fontweight='bold', color='darkgreen', pad=10)
        else:
            axes[1, 2].imshow(mask, cmap='gray', vmin=0, vmax=1)
            axes[1, 2].set_title('BCE+Dice ★\nBest shape & accuracy', fontsize=14,
                               fontweight='bold', color='darkgreen', pad=10)
        axes[1, 2].axis('off')

        for spine in axes[1, 2].spines.values():
            spine.set_edgecolor('darkgreen')
            spine.set_linewidth(5)
            spine.set_visible(True)

        if show_probability:
            cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.02])
            cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
            cbar.set_label('Prediction Probability', fontsize=14, fontweight='bold')

        img_name = os.path.basename(image_path)
        map_type = "Probability Maps" if show_probability else "Binary Masks"
        fig.suptitle(f'Loss Function Comparison - {map_type}\n{img_name}',
                    fontsize=18, fontweight='bold', y=0.98)

        plt.tight_layout(rect=[0, 0.08 if show_probability else 0, 1, 0.96])
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()


def main():
    checkpoint_paths = {
        'BCE': '/content/drive/MyDrive/ELEC_576/Final_Project/experiments/bce_20251130_065539/best_model.pth',
        'DICE': '/content/drive/MyDrive/ELEC_576/Final_Project/experiments/dice_20251130_081424/best_model.pth',
        'FOCAL': '/content/drive/MyDrive/ELEC_576/Final_Project/experiments/focal_20251130_093204/best_model.pth',
        'BCE_DICE': '/content/drive/MyDrive/ELEC_576/Final_Project/experiments/bce_dice_20251130_104932/best_model.pth'
    }

    generator = LandscapeComparisonGenerator(checkpoint_paths, img_size=224)

    test_image = '/content/plantsegv2/images/test/apple_black_rot_175.jpg'
    ground_truth = '/content/plantsegv2/annotations/test/apple_black_rot_175.png'

    generator.create_landscape_comparison(
        image_path=test_image,
        save_path='./comparison_landscape_probability.png',
        ground_truth_path=ground_truth,
        show_probability=True
    )

    generator.create_simple_plantseg_example(
        image_path=test_image,
        save_path='./plantseg_example.png',
        ground_truth_path=ground_truth
    )


if __name__ == '__main__':
    main()


Loading BCE model...
Loading pretrained ViT weights...
✓ Successfully loaded pretrained ViT weights!
  Loaded 146 pretrained parameters
Loaded from epoch 37, Val mIoU: 0.5453

Loading DICE model...
Loading pretrained ViT weights...
✓ Successfully loaded pretrained ViT weights!
  Loaded 146 pretrained parameters
Loaded from epoch 47, Val mIoU: 0.5589

Loading FOCAL model...
Loading pretrained ViT weights...
✓ Successfully loaded pretrained ViT weights!
  Loaded 146 pretrained parameters
Loaded from epoch 43, Val mIoU: 0.5380

Loading BCE_DICE model...
Loading pretrained ViT weights...
✓ Successfully loaded pretrained ViT weights!
  Loaded 146 pretrained parameters
Loaded from epoch 47, Val mIoU: 0.5632

All 4 models loaded successfully!


/tmp/ipython-input-2981172551.py:180: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0.08 if show_probability else 0, 1, 0.96])
/tmp/ipython-input-2981172551.py:180: UserWarning: Glyph 9733 (\N{BLACK STAR}) missing from font(s) DejaVu Serif.
  plt.tight_layout(rect=[0, 0.08 if show_probability else 0, 1, 0.96])
/tmp/ipython-input-2981172551.py:181: UserWarning: Glyph 9733 (\N{BLACK STAR}) missing from font(s) DejaVu Serif.
  plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')


Landscape comparison saved to: ./comparison_landscape_probability.png
Simple comparison saved to: ./plantseg_example.png
